In [1]:
from geopy.geocoders import Nominatim
import polars as pl
import plotly.express as px
from polars import col as c

In [2]:
df = pl.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vSvTrBNbcpx6fu7DBog8kbYiSJOYVwxP1cQHs_bBwiGYYIQglMAaY9Aw_tw4Rn0jswDN0vcAH-DS8A2/pub?output=csv', truncate_ragged_lines=True)
# Remove all the empty rows
df = df.filter(c('Quilt Name') != ' ')

In [ ]:
# Get all the latitude and longitudes
geolocator = Nominatim(user_agent="quilt_tracker")
locs = df['Location'].map_elements(geolocator.geocode)

In [129]:
df = df.with_columns(
    lat=locs.map_elements(lambda l: l.latitude, return_dtype=float),
    lon=locs.map_elements(lambda l: l.longitude, return_dtype=float),
    info=c('Quilt Show/Museum') + ' in ' + c('Location') + '<br>' + c('Date Arrived') + ' - ' + c('Date Left') + '<br>' + c('Additional Notes') + '<br><a href="' + c('Website') + '">Website</a>'
)

In [86]:
fig = px.scatter_geo(df,
    lat='lat',
    lon='lon',
    color='Quilt Name',
    hover_name='Quilt Name',
    hover_data={
        'Place': True,
        'Quilt Name': False,
        'lat': False,
        'lon': False,
    },
    title="Amy's quilts and where they've been. Hover over for more information",
)
fig.update_geos(
    # fitbounds='locations',
    # visible=False,
    # bgcolor='green',
    # coastlinecolor='blue',
    # countrycolor='orange',
    # coastlinewidth=5,
    # framecolor='blue',
    lakecolor='lightblue',
    # oceancolor='blue',
    # showocean=True,
    # showland=True,
    landcolor='#e9b7f1',
    subunitcolor="#888888",
    # projection_type="orthographic"
)
fig.update_layout(
    geo_scope='usa',
    height=500,
    margin={"r":0,"t":40,"l":0,"b":0},
    hovermode='x',
    # hoverdistance=100,
)

In [87]:
fig.write_html('graph.html')

'2024-09-05 14:53:08.305142'

In [88]:
%pip install gmplot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.7/164.7 kB 739.7 kB/s eta 0:00:00 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [131]:
import gmplot

# Create the map plotter:
# apikey = '' # (your API key here)
# gmap = gmplot.GoogleMapPlotter(37.766956, -122.448481, 14, apikey=apikey)
gmap = gmplot.GoogleMapPlotter(40.395124, -100.501961, 5, scale_control=True, title="Amy's quilts and where they've been")
# Outline the Golden Gate Park:
# golden_gate_park = zip(*[
#     (37.771269, -122.511015),
#     (37.773495, -122.464830),
#     (37.774797, -122.454538),
#     (37.771988, -122.454018),
#     (37.773646, -122.440979),
#     (37.772742, -122.440797),
#     (37.771096, -122.453889),
#     (37.768669, -122.453518),
#     (37.766227, -122.460213),
#     (37.764028, -122.510347)
# ])
# gmap.polygon(*golden_gate_park, color='cornflowerblue', edge_width=10)

# Highlight some attractions:
# attractions_lats, attractions_lngs = zip(*[
    # (37.769901, -122.498331),
    # (37.768645, -122.475328),
    # (37.771478, -122.468677),
    # (37.769867, -122.466102),
    # (37.767187, -122.467496),
    # (37.770104, -122.470436)
# ])
# gmap.scatter(df['lat'].to_list(), df['lon'].to_list(),
#     color='blue',
#     marker=True,
#     title=df['Place'].to_list(),
#     label=df['Quilt Name'].to_list(),
#     alpha=.2,
# )
for i in df.iter_rows(named=True):
    gmap.marker(i['lat'], i['lon'],
        title=i['Quilt Name'],
        label=i['Quilt Name'],
        info_window=i['info'],
        draggable=False,
    )
# Mark a hidden gem:
# gmap.marker(37.770776, -122.461689, color='cornflowerblue')

# Draw the map:
gmap.draw('map.html')

[43.4918307, 31.081224383093396, 43.6086295]